<a href="https://colab.research.google.com/github/GajendraIITB/Projects-/blob/Codes/Dating_Recommendations_using_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Let’s get started with the task of dating recommendations by importing the necessary Python libraries and the dataset:

In [3]:
import pandas as pd
import plotly.express as px

data = pd.read_csv("/content/gdrive/My Drive/dating_app_dataset.csv")
print(data.head())

   User ID  Age  Gender    Height  \
0        1   30    Male  5.240385   
1        2   27  Female  4.937625   
2        3   29  Female  5.806296   
3        4   29  Female  5.101402   
4        5   32    Male  5.986670   

                                           Interests             Looking For  \
0  ['Sports', 'Cooking', 'Hiking', 'Music', 'Movi...           Casual Dating   
1                              ['Sports', 'Reading']              Friendship   
2                                         ['Sports']           Casual Dating   
3                                        ['Reading']                Marriage   
4  ['Sports', 'Hiking', 'Music', 'Movies', 'Readi...  Long-term Relationship   

  Children    Education Level               Occupation  Swiping History  \
0       No        High School                  Student               96   
1      Yes    Master's Degree                   Artist               96   
2       No  Bachelor's Degree  Social Media Influencer               64

Now, let’s explore this data before creating a dating recommendation system. I’ll start with exploring the age distribution in the data by gender:

In [4]:
# age distribution by gender
fig = px.histogram(data, x="Age", color="Gender", nbins=20,
                   title="Age Distribution by Gender")
fig.update_layout(xaxis_title="Age", yaxis_title="Count")
fig.show()

Now, let’s have a look at the education level distribution by gender:

In [5]:
# education level distribution by gender
education_order = ["High School", "Bachelor's Degree", "Master's Degree", "Ph.D."]
fig = px.bar(data, x="Education Level", color="Gender",
             category_orders={"Education Level": education_order},
             title="Education Level Distribution by Gender")

fig.update_layout(xaxis_title="Education Level", yaxis_title="Count")
fig.show()

Now, let’s have a look at the frequency of app usage by gender

In [6]:
# frequency of app usage distribution
fig = px.bar(data, x="Frequency of Usage",
             title="Frequency of App Usage Distribution")
fig.update_layout(xaxis_title="Frequency of Usage",
                  yaxis_title="Count")
fig.show()

Now, I’ll separate data into male and female profiles:

In [7]:
# Separate data into male and female
male_profiles = data[data['Gender'] == 'Male']
female_profiles = data[data['Gender'] == 'Female']

Now, let’s calculate the match score for dating recommendations:

In [8]:
def calculate_match_score(profile1, profile2):
    # Shared interests score (1 point per shared interest)
    interests1 = set(eval(profile1['Interests']))
    interests2 = set(eval(profile2['Interests']))
    shared_interests_score = len(interests1.intersection(interests2))

    # Age difference score (higher age difference, lower score)
    age_difference_score = max(0, 10 - abs(profile1['Age'] - profile2['Age']))

    # Swiping history score (higher swiping history, higher score)
    swiping_history_score = min(profile1['Swiping History'], profile2['Swiping History']) / 100

    # Relationship type score (1 point for matching types)
    relationship_type_score = 0
    if profile1['Looking For'] == profile2['Looking For']:
        relationship_type_score = 1
    # Total match score
    total_score = (
        shared_interests_score + age_difference_score + swiping_history_score + relationship_type_score
    )

    return total_score

# Example: Calculate match score between two profiles
profile1 = male_profiles.iloc[0]
profile2 = female_profiles.iloc[0]
match_score = calculate_match_score(profile1, profile2)
print(f"Match score between User {profile1['User ID']} and User {profile2['User ID']} : {match_score}")

Match score between User 1 and User 2 : 9.96


In the above code, we defined a function called calculate_match_score that calculates a compatibility score between two user profiles on a dating app. The goal is to quantify how well two users might match based on various factors. Let’s go through the steps we covered in the above code to calculate the match score:

1. We started by considering the interests or hobbies of both users listed in their profiles. Each shared interest contributes 1 point to the compatibility score. The more shared interests they have, the higher their compatibility score will be. For example, if both users enjoy travelling, that’s a shared interest and adds a point to their score.

2. Age can play a significant role in compatibility. We calculated a score based on the age difference between the two users. If the age difference is small (within ten years), they receive a higher score. However, if the age difference is large, the score starts decreasing. It ensures that users with similar ages are more likely to have a higher match score.

3. User engagement with the app, as measured by their swiping history (e.g., the number of likes or dislikes they’ve given), is considered. A higher swiping history indicates more active use of the app to find a partner and contributes to a higher score. It encourages matchmaking with users who are actively engaged in the dating process.

4. The type of relationship each user is looking for (e.g., long-term relationship, marriage) is also used to calculate the match score. If both users are seeking the same type of relationship, they receive a point. This score reflects the compatibility of their relationship goals.

5. The final compatibility score is calculated by summing up the individual scores from the previous steps. A higher total score suggests a better potential match, while a lower score indicates a less compatible match.


Now, here’s how to generate dating recommendations by utilizing the match score we calculated above:

In [9]:
def recommend_profiles(male_profiles, female_profiles):
    recommendations = []

    for _, male_profile in male_profiles.iterrows():
        best_match = None
        best_score = -1

        for _, female_profile in female_profiles.iterrows():
            score = calculate_match_score(male_profile, female_profile)

            if score > best_score:
                best_match = female_profile
                best_score = score

        recommendations.append((male_profile, best_match, best_score))

    return recommendations


# Generate recommendations
recommendations = recommend_profiles(male_profiles, female_profiles)

# Sort recommendations by match score in descending order
recommendations.sort(key=lambda x: x[2], reverse=True)

# Display the top recommendations
for idx, (male_profile, female_profile, match_score) in enumerate(recommendations[:10]):
    print(f"Recommendation {idx + 1}:")
    print(f"Male Profile (User {male_profile['User ID']}): Age {male_profile['Age']}, Interests {male_profile['Interests']}")
    print(f"Female Profile (User {female_profile['User ID']}): Age {female_profile['Age']}, Interests {female_profile['Interests']}")
    print(f"Match Score: {match_score}")
    print()


Recommendation 1:
Male Profile (User 36): Age 19, Interests ['Movies', 'Cooking', 'Hiking', 'Reading', 'Sports', 'Travel', 'Music']
Female Profile (User 451): Age 19, Interests ['Reading', 'Music', 'Cooking', 'Hiking', 'Travel', 'Sports', 'Movies']
Match Score: 18.79

Recommendation 2:
Male Profile (User 274): Age 29, Interests ['Reading', 'Movies', 'Travel', 'Music', 'Hiking', 'Cooking', 'Sports']
Female Profile (User 300): Age 29, Interests ['Cooking', 'Reading', 'Music', 'Hiking', 'Travel', 'Sports', 'Movies']
Match Score: 18.73

Recommendation 3:
Male Profile (User 456): Age 29, Interests ['Cooking', 'Hiking', 'Sports', 'Travel', 'Music', 'Movies', 'Reading']
Female Profile (User 65): Age 29, Interests ['Travel', 'Movies', 'Reading', 'Sports', 'Music', 'Cooking', 'Hiking']
Match Score: 18.69

Recommendation 4:
Male Profile (User 147): Age 34, Interests ['Reading', 'Travel', 'Movies', 'Hiking', 'Cooking', 'Music', 'Sports']
Female Profile (User 287): Age 34, Interests ['Reading', 'H

In the above code, we are creating a recommendation system that suggests potential matches between male and female user profiles. The recommendation system evaluates the compatibility of these profiles based on various factors, including shared interests, age, and other attributes we used to calculate the match score. Let’s go through the steps covered in this code section one by one:

1. The function defined above is responsible for generating recommendations. It takes two sets of user profiles as input, one for males and one for females, and returns a list of recommended matches.

2. It iterates through each male user profile one by one. For each male profile, it searches through the female profiles to find the best potential match. It initially sets best_match to None and best_score to a negative value to ensure that the first female profile encountered becomes the initial best match.

3. As the code iterates through female profiles, it calculates the match score for each male-female profile pair and compares it to the previous best score. If the new score is higher, it updates the best_match and best_score with the current female profile and score.

4. Once the best match is found for a male profile, it is stored as a recommendation in the recommendations list, along with the male and female profiles involved in the match and the match score.

5. After generating recommendations for all male profiles, the code sorts the recommendations based on the match score in descending order. It ensures that the highest-scoring recommendations appear at the top.

6. Finally, it prints out the top recommendations, including information about the male and female profiles and their match scores.

So, This is how we can generate dating recommendations for matchmaking of profiles on a dating app using Python.